# MINST Dataset

..Using KNeighborsClassifier

In [0]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

## Getting the Dataset

In [0]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'DESCR', 'details', 'categories', 'url'])

In [0]:
X, y = mnist['data'], mnist['target']

Convert labels from string to integer

In [0]:
y = y.astype(np.uint8)

In [0]:
def plot_digit(digit):
  digit_img = digit.reshape(28, 28)
  plt.imshow(digit_img, cmap="binary")
  plt.axis("off")
  plt.show()

## Splitting the Data

MNIST dataset is already split into training and test set

In [0]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

## Preprocessing Pipeline

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

preprocessing = Pipeline([('standardization', StandardScaler())])

## Hyperparameter search

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
param_grid = [{'weights': ['uniform', 'distance'], 'n_neighbors': [3, 4]}]
kn_clf = GridSearchCV(kn, param_grid, verbose=3)
X_train_prepared = preprocessing.fit_transform(X_train, y_train)
kn_clf.fit(X_train_prepared, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] n_neighbors=3, weights=uniform ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... n_neighbors=3, weights=uniform, score=0.942, total=15.4min
[CV] n_neighbors=3, weights=uniform ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 15.4min remaining:    0.0s


[CV] ...... n_neighbors=3, weights=uniform, score=0.944, total=15.3min
[CV] n_neighbors=3, weights=uniform ..................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 30.7min remaining:    0.0s


[CV] ...... n_neighbors=3, weights=uniform, score=0.945, total=15.4min
[CV] n_neighbors=3, weights=uniform ..................................
[CV] ...... n_neighbors=3, weights=uniform, score=0.938, total=15.4min
[CV] n_neighbors=3, weights=uniform ..................................
[CV] ...... n_neighbors=3, weights=uniform, score=0.945, total=15.4min
[CV] n_neighbors=3, weights=distance .................................
[CV] ..... n_neighbors=3, weights=distance, score=0.942, total=15.4min
[CV] n_neighbors=3, weights=distance .................................
[CV] ..... n_neighbors=3, weights=distance, score=0.946, total=15.3min
[CV] n_neighbors=3, weights=distance .................................
[CV] ..... n_neighbors=3, weights=distance, score=0.945, total=15.4min
[CV] n_neighbors=3, weights=distance .................................
[CV] ..... n_neighbors=3, weights=distance, score=0.941, total=15.3min
[CV] n_neighbors=3, weights=distance .................................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 308.8min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [3, 4],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [0]:
kn_clf.best_params_

{'n_neighbors': 4, 'weights': 'distance'}